In [ ]:
!pip install tsai

需要修改一下/usr/local/lib/python3.7/dist-packages/tsai-0.2.26-py3.7.egg/tsai/models/utils.py这里的内容。添加一个NewModel或者之类的东西。再次手动增加内容。
后续可以考虑添加对应的包，直接在安装的时候成功。
同时如果针对LSTM-FCN，需要添加一个参数输入的内容

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Thesis_exp')
!ls

In [ ]:
from tsai.all import *
import os
computer_setup()


In [ ]:
# coding:UTF-8
# V2, ResNet Imporve
# V2.1, Change the built_layer method. 


from tsai.imports import *
from tsai.utils import *
from torch.nn.init import normal_
from fastai.torch_core import Module
from fastai.layers import *
from torch.nn.utils import weight_norm, spectral_norm

Convolution = [8, 16, 32, 64, 128, 256]
# ZeroOp, 默认
# MaxPooling, 默认
# Dropout, 0-0.5之间
# Activate = {1: 'softmax', 2: 'elu', 3: 'Relu', 4: 'selu', 5: 'sigmoid'}
Activate = ['softmax', 'elu', 'Relu', 'selu', 'sigmoid']
Dense = [8, 16, 32, 64, 128, 256]



from tsai.imports import *
from tsai.utils import *
from torch.nn.init import normal_
from fastai.torch_core import Module
from fastai.layers import *
from torch.nn.utils import weight_norm, spectral_norm


# DenseBlock
class DenseBlock(nn.Sequential):
    "Create a sequence of conv1d (`ni` to `nf`), activation (if `act_cls`) and `norm_type` layers."
    def __init__(self, cin, cout, **kwargs):
        layers = []
        layers += nn.Linear(cin, cout)
        layers += nn.ReLU()
        super().__init__(*layers)
        # 找到此时ConvBlock的父类Sequential, 随后调用它的初始化函数



class NewModel(Module):
    def __init__(self, c_in, c_out, seq_len=None, hidden_size=100, rnn_layers=1, bias=True, cell_dropout=0,
                 rnn_dropout=0.8, bidirectional=False, shuffle=True,
                 fc_dropout=0., conv_layers=[128, 256, 128], kss=[7, 5, 3], se=0, Phenotype=[]):
        if shuffle: assert seq_len is not None, 'need seq_len if shuffle=True'

        # RNN
        front_output = conv_layers[2]
        self.rnn = nn.LSTM(seq_len if shuffle else c_in, hidden_size, num_layers=rnn_layers, bias=bias,
                              batch_first=True,
                              dropout=cell_dropout, bidirectional=bidirectional)
        self.rnn_dropout = nn.Dropout(rnn_dropout) if rnn_dropout else noop
        self.shuffle = Permute(0, 2,
                               1) if not shuffle else noop  # You would normally permute x. Authors did the opposite.
        print(seq_len)
        # FCN，主要解决下方cnn部分的内容
        assert len(conv_layers) == len(kss)

        # 卷积之后，再加一层池化、激活函数
        self.convblock1 = ConvBlock(c_in, conv_layers[0], kss[0])
        self.se1 = SqueezeExciteBlock(conv_layers[0], se) if se != 0 else noop

        self.convblock2 = ConvBlock(conv_layers[0], conv_layers[1], kss[1])
        self.se2 = SqueezeExciteBlock(conv_layers[1], se) if se != 0 else noop

        self.convblock3 = ConvBlock(conv_layers[1], conv_layers[2], kss[2])

        # Added_layer
        self.layer_list = []
        # print(len(Phenotype))
        for i in range(len(Phenotype)):
            layer, front_output = self._make_layer(Phenotype[i], front_output)
            # 如果ZeroOP
            if layer == 0:
                continue
            self.layer_list.append(layer)
        
        # print(self.layer_list)

        self.net = nn.Sequential(*self.layer_list)

        self.gap = GAP1d(1)

        # Common
        self.concat = Concat()
        self.fc_dropout = nn.Dropout(fc_dropout) if fc_dropout else noop
        self.fc = nn.Linear(hidden_size * (1 + bidirectional) + front_output, c_out)
        

    def forward(self, x):
        # RNN
        rnn_input = self.shuffle(x)  # permute --> (batch_size, seq_len, n_vars) when batch_first=True
        output, _ = self.rnn(rnn_input)
        last_out = output[:, -1]  # output of last sequence step (many-to-one)
        last_out = self.rnn_dropout(last_out)

        # FCN
        x = self.convblock1(x)
        x = self.se1(x)
        x = self.convblock2(x)
        x = self.se2(x)
        x = self.convblock3(x)
        # print("Befor New Layers: ", x.shape)

        # x = self.net(x)
        for operator in self.net:
            if isinstance(operator, nn.Linear):
                x = torch.transpose(x, 1, 2)
                x = operator(x)
                x = torch.transpose(x, 1, 2)
            else:
                x = operator(x)
            # print(f"After a New Layer {operator}:", x.shape)
        
        x = self.gap(x)

        # Concat
        x = self.concat([last_out, x])
        x = self.fc_dropout(x)
        x = self.fc(x)

        return x
    
    
    def _make_layer(self, Phenotype, front_output):
        # Make layer by Phenotype
        if Phenotype[0] == 0:  # conv
            out_size = Convolution[int(Phenotype[1])]            
            return ConvBlock(front_output, out_size, 3), out_size
        
        elif Phenotype[0] == 1:  # ZeroOp
            return 0, front_output

        elif Phenotype[0] == 2:  # Maxpooling, Pooling size = 3, zero padding.
            return nn.MaxPool1d(kernel_size=3, stride=1, padding=1), front_output

        elif Phenotype[0] == 3:  # Dropout
            dropout_rate = Phenotype[1]
            return nn.Dropout(dropout_rate), front_output

        elif Phenotype[0] == 4: # Activate
            if Phenotype[1] == 0:
                act = nn.Softmax()
            elif Phenotype[1] == 1:
                act = nn.ELU()
            elif Phenotype[1] == 2:
                act = nn.ReLU()
            elif Phenotype[1] == 3:
                act = nn.SELU()
            elif Phenotype[1] == 4:
                act = nn.Sigmoid()
            return act, front_output

        elif Phenotype[0] == 5: # FC
            out_size = Dense[int(Phenotype[1])]
            return nn.Linear(front_output, out_size), out_size



In [ ]:
# coding:UTF-8
# V2, LSTM-FCN Imporve
# 3.11.2022 run data, GPU

from __future__ import division
from numpy import *
import random as rd
import pandas as pd
import pickle

NP = 10
size = 15
xMin = -10
xMax = 10
F = 0.8
CR = 0.5
LAYER_THRESHOLD = 0.9
GEN_MAX = 10
TRAIN_EPOCHES = 100

# 神经结构的参数设置
Convolution = [8, 16, 32, 64, 128, 256]
# ZeroOp, 默认
# MaxPooling, 默认
# Dropout, 0-0.5之间
# Activate = {1: 'softmax', 2: 'elu', 3: 'Relu', 4: 'selu', 5: 'sigmoid'}
Activate = ['softmax', 'elu', 'Relu', 'selu', 'sigmoid']
Dense = [8, 16, 32, 64, 128, 256]



# all_dataset = ['ACSF1', 'Adiac', 'AllGestureWiimoteX', 'AllGestureWiimoteY', 'AllGestureWiimoteZ', 
#                'ArrowHead', 'Beef', 'BeetleFly', 'BirdChicken', 'BME', 'Car', 'CBF', 'Chinatown', 
#                'ChlorineConcentration', 'CinCECGTorso', 'Coffee', 'Computers', 'CricketX', 'CricketY', 
#                'CricketZ', 'Crop', 'DiatomSizeReduction', 'DistalPhalanxOutlineAgeGroup', 
#                'DistalPhalanxOutlineCorrect', 'DistalPhalanxTW', 'DodgerLoopDay', 'DodgerLoopGame', 
#                'DodgerLoopWeekend', 'Earthquakes', 'ECG200', 'ECG5000', 'ECGFiveDays', 'ElectricDevices', 
#                'EOGHorizontalSignal', 'EOGVerticalSignal', 'EthanolLevel', 'FaceAll', 'FaceFour', 'FacesUCR', 
#                'FiftyWords', 'Fish', 'FordA', 'FordB', 'FreezerRegularTrain', 'FreezerSmallTrain', 'Fungi', 
#                'GestureMidAirD1', 'GestureMidAirD2', 'GestureMidAirD3', 'GesturePebbleZ1', 'GesturePebbleZ2', 
#                'GunPoint', 'GunPointAgeSpan', 'GunPointMaleVersusFemale', 'GunPointOldVersusYoung', 'Ham', 
#                'HandOutlines', 'Haptics', 'Herring', 'HouseTwenty', 'InlineSkate', 'InsectEPGRegularTrain', 
#                'InsectEPGSmallTrain', 'InsectWingbeatSound', 'ItalyPowerDemand', 'LargeKitchenAppliances', 
#                'Lightning2', 'Lightning7', 'Mallat', 'Meat', 'MedicalImages', 'MelbournePedestrian', 
#                'MiddlePhalanxOutlineAgeGroup', 'MiddlePhalanxOutlineCorrect', 'MiddlePhalanxTW', 
#                'MixedShapesRegularTrain', 'MixedShapesSmallTrain', 'MoteStrain', 'NonInvasiveFetalECGThorax1', 
#                'NonInvasiveFetalECGThorax2', 'OliveOil', 'OSULeaf', 'PhalangesOutlinesCorrect', 'Phoneme', 
#                'PickupGestureWiimoteZ', 'PigAirwayPressure', 'PigArtPressure', 'PigCVP', 'PLAID', 'Plane', 
#                'PowerCons', 'ProximalPhalanxOutlineAgeGroup', 'ProximalPhalanxOutlineCorrect', 'ProximalPhalanxTW', 
#                'RefrigerationDevices', 'Rock', 'ScreenType', 'SemgHandGenderCh2', 'SemgHandMovementCh2', 
#                'SemgHandSubjectCh2', 'ShakeGestureWiimoteZ', 'ShapeletSim', 'ShapesAll', 'SmallKitchenAppliances', 
#                'SmoothSubspace', 'SonyAIBORobotSurface1', 'SonyAIBORobotSurface2', 'StarLightCurves', 'Strawberry', 
#                'SwedishLeaf', 'Symbols', 'SyntheticControl', 'ToeSegmentation1', 'ToeSegmentation2', 'Trace', 
#                'TwoLeadECG', 'TwoPatterns', 'UMD', 'UWaveGestureLibraryAll', 'UWaveGestureLibraryX', 
#                'UWaveGestureLibraryY', 'UWaveGestureLibraryZ', 'Wafer', 'Wine', 'WordSynonyms', 'Worms', 
#                'WormsTwoClass', 'Yoga']


# 完成的数据集: 'ACSF1', 'Adiac', 'ArrowHead', 'Beef', 'BeetleFly', 'BirdChicken', 'BME', 'Car', 'CBF', 'Chinatown', 'ChlorineConcentration', 
#             'CinCECGTorso', 'Coffee', 'Computers', 'CricketX', 'CricketY', 'CricketZ', 'DiatomSizeReduction', 'DistalPhalanxOutlineAgeGroup', 
#             'DistalPhalanxOutlineCorrect', 'DistalPhalanxTW','Earthquakes', 'ECG200', 'ECG5000', 'ECGFiveDays', 'EOGHorizontalSignal', 
#             'EOGVerticalSignal', 'EthanolLevel','FaceAll', 'FaceFour', 'FacesUCR', 'FiftyWords', 'Fish', 'FordA', 'FreezerRegularTrain', 'FreezerSmallTrain', 'Fungi', 
#             'GunPoint','GunPointAgeSpan', 'GunPointMaleVersusFemale', 'GunPointOldVersusYoung', 'Ham', 
#             'HandOutlines', 'Haptics', 'Herring', 'HouseTwenty', 'InlineSkate', 'InsectEPGRegularTrain', 'InsectEPGSmallTrain', 'InsectWingbeatSound', 'ItalyPowerDemand', 'LargeKitchenAppliances',             
#             'Lightning2', 'Lightning7', 'Mallat', 'Meat', 'MedicalImages',  'MiddlePhalanxOutlineAgeGroup', 'MiddlePhalanxOutlineCorrect',  'MiddlePhalanxTW',  
#             'MixedShapesRegularTrain', 'MixedShapesSmallTrain', 'MoteStrain', 'NonInvasiveFetalECGThorax1','OliveOil', 'OSULeaf', 'PhalangesOutlinesCorrect', 'Phoneme', 
#             'PigAirwayPressure', 'PigArtPressure', 'PigCVP',  'Plane', 'PowerCons', 'ProximalPhalanxOutlineAgeGroup', 'ProximalPhalanxOutlineCorrect', 'ProximalPhalanxTW', 
#             'RefrigerationDevices', 'Rock', 'ScreenType', 'SemgHandGenderCh2', 'SemgHandMovementCh2', 'SemgHandSubjectCh2', 
#             'ShapeletSim', 'ShapesAll', 'SmallKitchenAppliances', 'SmoothSubspace', 'SonyAIBORobotSurface1', 'SonyAIBORobotSurface2', 'Strawberry', 
#             'SwedishLeaf', 'Symbols', 'SyntheticControl', 'ToeSegmentation1', 'ToeSegmentation2', 'Trace', 'TwoLeadECG', 'TwoPatterns', 'UMD',
#             'UWaveGestureLibraryX', 'UWaveGestureLibraryY', 'UWaveGestureLibraryZ', 'Wafer', 'Wine', 'WordSynonyms', 'Worms', 'WormsTwoClass', 'Yoga'
#             'AllGestureWiimoteX', 'AllGestureWiimoteY', 'AllGestureWiimoteZ', 'DodgerLoopDay', 'DodgerLoopGame', 'DodgerLoopWeekend', 
#             'GestureMidAirD1', 'GestureMidAirD2', 'GestureMidAirD3', 'GesturePebbleZ1', 'GesturePebbleZ2', 'PickupGestureWiimoteZ', 'PLAID',
#             'ShakeGestureWiimoteZ', 'MelbournePedestrian',            
#             'Crop', 'ElectricDevices', 'FordB', 'NonInvasiveFetalECGThorax2', 'StarLightCurves', 'UWaveGestureLibraryAll', 
#             
#             
# 异常数据集:    'AllGestureWiimoteX', 'AllGestureWiimoteY', 'AllGestureWiimoteZ', 'DodgerLoopDay', 'DodgerLoopGame', 'DodgerLoopWeekend', 
#              'GestureMidAirD1', 'GestureMidAirD2', 'GestureMidAirD3', 'GesturePebbleZ1', 'GesturePebbleZ2', 'PickupGestureWiimoteZ', 'PLAID',
#              'ShakeGestureWiimoteZ', 'MelbournePedestrian',
#           
# 运算比较久的数据集: 'Crop', 'ElectricDevices', 'FordB', 'NonInvasiveFetalECGThorax2', 'StarLightCurves', 'UWaveGestureLibraryAll', 


dataset_with_nan = ['AllGestureWiimoteX', 'AllGestureWiimoteY', 'AllGestureWiimoteZ', 'DodgerLoopDay', 'DodgerLoopGame', 'DodgerLoopWeekend', 
             'GestureMidAirD1', 'GestureMidAirD2', 'GestureMidAirD3', 'GesturePebbleZ1', 'GesturePebbleZ2', 'PickupGestureWiimoteZ', 'PLAID',
             'ShakeGestureWiimoteZ', 'MelbournePedestrian',]


# all_dataset = ['Crop', 'ElectricDevices', 'FordB', 'NonInvasiveFetalECGThorax2', 'StarLightCurves', 'UWaveGestureLibraryAll']
all_dataset = ['Crop']


# 优点：可以并行计算，可以模拟nature中的进化规律，获得constraint 之外的结构


# 翻译此时的结构
def translate_architecture(Individual, display=False):
    m, n = shape(Individual)
    Actual_structure = zeros((m, n))
    layers = []
    # 15 * 2
    for i in range(m):
        # Conv layer
        if Individual[i][0] == 0:
            single_layer = ["conv"]
            single_layer.append(Convolution[int(Individual[i][1] * 6)])
            Actual_structure[i] = [Individual[i][0], int(Individual[i][1] * 6)]
        # ZeroOp layer
        elif Individual[i][0] == 1:
            single_layer = ["ZeroOp"]
            single_layer.append("Default")
            Actual_structure[i] = [Individual[i][0], Individual[i][1]]
        # Maxpooling layer
        elif Individual[i][0] == 2:
            single_layer = ["Maxpooling"]
            single_layer.append("Default")
            Actual_structure[i] = [Individual[i][0], Individual[i][1]]
        # Dropout layer
        elif Individual[i][0] == 3:
            single_layer = ["Dropout"]
            single_layer.append(0.5*Individual[i][1])
            Actual_structure[i] = [Individual[i][0], 0.5*Individual[i][1]]
        # Activate layer
        elif Individual[i][0] == 4:
            single_layer = ["Activate"]
            single_layer.append(Activate[int(Individual[i][1] * 5)])
            Actual_structure[i] = [Individual[i][0], int(Individual[i][1] * 5)]
        # Dense layer
        elif Individual[i][0] == 5:
            single_layer = ["Dense"]
            single_layer.append(Dense[int(Individual[i][1] * 6)])
            Actual_structure[i] = [Individual[i][0], int(Individual[i][1] * 6)]
        layers.append(single_layer)
    if display:
        print(f"structure is {layers}")
    return Actual_structure


def checkifweak(Individual):
    # 判断此时是否已经是弱基因
    # Add basic constraint

    m, n = shape(Individual)
    for i in range(m - 1):
        # 如果此时是两个连续的下采样就返回true
        if Individual[i][0] == 2 and Individual[i + 1][0] == 2:
            return True
        # 如果此时是两个连续的激活函数就返回True
        elif Individual[i][0] == 4 and Individual[i + 1][0] == 4:
            return True
        # 如果此时是两个相邻的FCN，同时有一个降维特别大，就返回True
        elif Individual[i][0] == 5 and Individual[i + 1][0] == 5:
            if Individual[i][1] < 0.5 or Individual[i + 1][1] < 0.5:
              return True
    return False


# 通过此时的基因型获取模型
def GetModel(Phenotype, dls):    
    m, n = shape(Phenotype)
    arch = NewModel
    model = create_model(arch, dls=dls, Phenotype=Phenotype)
    # arch = LSTM_FCN
    # model = create_model(arch, dls=dls)
    print(arch.__name__)
    print(model)
    return model


# 获取单个个体的基因型对应的适应度(Pred)
def getPred(Phenotype, dls):
    
    # 训练
    model = GetModel(Phenotype, dls)
    
    learn = Learner(dls, model,  metrics=accuracy)
    start = time.time()
    learn.fit_one_cycle(TRAIN_EPOCHES, 1e-3)
    elapsed = time.time() - start
    vals = learn.recorder.values[-1]
    print(f"训练耗时:{int(elapsed)}")
    # vals 的结构：val[3] = ["train loss", "valid loss", "accuracy"]
    return vals[2]
    # return 1


# 计算适应值函数
def calFitness(Individual, dls):
    # Translate Individual Architecture
    # 目标: pred
    # Phenotype = translate_architecture(Individual, display=True)
    Phenotype = translate_architecture(Individual)
    fitness = getPred(Phenotype, dls)
    return fitness


# 变异函数
def mutation(X_Parent, F, LAYER_THRESHOLD):
    # Mutate for Neural network
    print(shape(X_Parent))
    m, n, k = shape(X_Parent)
    # print()
    XMutationTmp = zeros((m, n, k))

    # 对于此时来说，需要额外设定一个layer_rate, 如果大于layer_rate，那么就mutate，layer，如果小于layer_rate，调整parameter。

    count = 0

    for i in range(m):
      # 需要调整，此时的r1应该删掉，随后改成其他的
        r1 = 0
        r2 = 0
        r3 = 0
        while r1 == i or r2 == i or r3 == i or r1 == r2 or r1 == r3 or r2 == r3:
            r1 = rd.randint(0, m - 1)
            r2 = rd.randint(0, m - 1)
            r3 = rd.randint(0, m - 1)

        for j in range(n):
            # 如果需要修改layer
            layer_change = rd.random()
            if (layer_change > LAYER_THRESHOLD):
                changed_layer = rd.randint(0, 5)
                XMutationTmp[i, j] = [changed_layer, rd.random()]
                continue

            # 如果不需要修改layer， 这里会有点担心会产生越界

            # 首先要看一下此时是不是同一类的r1, r2, r3，因为真的太严格了，所以就是说mutate也要考虑基因突变
            # 如果r1、r2都满足的话，就可以按照操作开始变换。
            if X_Parent[r1, j, 0] == X_Parent[r2, j, 0] or X_Parent[r1, j, 0] == X_Parent[r3, j, 0]:
                # 用6完成一次取余操作
                XMutationTmp[i, j] = [X_Parent[r1, j, 0], (X_Parent[r1, j, 1] + F * (X_Parent[r2, j, 1] - X_Parent[r3, j, 1])) % 1]
                # print("差分操作完成", count)
                # count += 1
            else:
                XMutationTmp[i, j] = X_Parent[i, j]
    # print()
    return XMutationTmp


# 交叉函数
def crossover(X_Parent, XMutationTmp, CR):
    # Binomial strategy
    # 按照CR crossrate完成修正增

    m, n, k = shape(X_Parent)
    X_Crossover = zeros((m, n, k))

    for i in range(m):
        for j in range(n):
            r = rd.random()
            if r <= CR:
                X_Crossover[i, j] = XMutationTmp[i, j]
            else:
                X_Crossover[i, j] = X_Parent[i, j]
    return X_Crossover


# 选择函数
def selection(X_Parent, XCrossover, fitnessVal, dls):
    # 从母代和子代中选取最高适应度的个体
    # 同时增加constraint机制，如果出现了低性状变异，就不需要测试其适应度

    m, n, k = shape(X_Parent)
    print("Parent_fitness:", fitnessVal)
    fitnessCrossOverVal = zeros((m, 1))
    for i in range(m):
        print(f"第{i + 1}个变异个体")

        if checkifweak(XCrossover[i]):
            print("子代个体基因型受限")
            print("Parent Better")
            continue

        fitnessCrossOverVal[i, 0] = calFitness(XCrossover[i], dls)
        if (fitnessCrossOverVal[i, 0] > fitnessVal[i, 0]):
            print("Mutate better")
            for j in range(n):
                X_Parent[i, j] = XCrossover[i, j]
            fitnessVal[i, 0] = fitnessCrossOverVal[i, 0]
        else:    
            print("Parent Better")
    return X_Parent, fitnessVal


# 保存最佳函数
def saveBest(XTemp, fitnessVal, round, dataset, dls):
    m = shape(fitnessVal)[0]
    print("After selection", fitnessVal)
    tmp = 0
    for i in range(1, m):
        if (fitnessVal[tmp] < fitnessVal[i]):
            tmp = i


    print("Round Best:")
    print(fitnessVal[tmp][0])
    best_individual_phenotype = translate_architecture(XTemp[tmp], display=True)

    model = GetModel(best_individual_phenotype, dls)
    
    if not os.path.exists("./model"):
        os.makedirs("./model") # 如果不存在目录figure_save_path，则创建

    torch.save(model, f"./model/{dataset}_{round}_{fitnessVal[tmp][0]}.pkl")

    if not os.path.exists("./genotype"):
        os.makedirs("./genotype") # 如果不存在目录figure_save_path，则创建

    np.save(f"./genotype/{dataset}_{round}_{fitnessVal[tmp][0]}.pkl", XTemp[tmp], allow_pickle=True)
    
    return fitnessVal[tmp][0]


# Differential_Evolution_Algorithm
def DEA(dsid, dls):
    # 初始化
    XTemp = zeros((NP, size, 2))
    # 构造对应的NP = 50, 基因长度size = 15, 每个size里包含的一个超参数，长度为2

    for i in range(NP):
        
        while(1):
            for j in range(size):
                # 初始化规则：对于每个XTemp[i, j]，[x, y]相当于初始化此时的类型x以及参数y
                # Conv的类型:          0; Conv的参数: [8, 16, 32, 64, 128, 256]; 一共6个参数，在乘以对应比例后截取整数部
                # ZeroOP的类型:        1; 没有参数
                # Maxpooling的类型:    2; MaxPooling，没有参数
                # Dropout的类型:       3; Dropout的参数: [0, 0.5]; random后乘以0.5
                # Activate的类型:      4; Activate的参数: ['softmax', 'elu', 'Relu', 'selu', 'sigmoid']; 5个参数，random后乘比例
                # Dense的类型:         5; Dense的参数: [4, 16, 32, 64, 128, 256]; 6个参数, random后乘比例。
                type = rd.randint(0, 5)
                XTemp[i, j] = [type, rd.random()]              
            if checkifweak(XTemp[i]):
                print("初始化不满足约束，重置当前基因型")
            else:
                break
        
    print("完成初始化")
    print(XTemp)
    

    if not os.path.exists("./generation"):
        os.makedirs("./generation") # 如果不存在目录figure_save_path，则创建
        
    for individual in range(NP):
        np.save(f"./generation/{dsid}_0_{individual}.pkl", XTemp[individual], allow_pickle=True)
        print(f"保存成功 ./generation/{dsid}_0_{individual}.pkl")


    print("完成种群记录")

    
    
    # 计算初始适应值
    fitnessVal = zeros((NP, 1))
    for i in range(NP):
        print(f"第{i + 1}个个体")
        fitnessVal[i, 0] = calFitness(XTemp[i], dls)

    acc_map = []
    fitness_for_record = np.empty(shape=(NP, 1))

    gen = 1
    while gen <= GEN_MAX:
        print("")
        print(f"Generation {gen}")
        
        # mutate完成
        XMutationTmp = mutation(XTemp, F, LAYER_THRESHOLD)
        print("Mutate 完成")
        # translate_architecture(XMutationTmp[0], display=True)

        print("Crossover完成")
        XCrossover = crossover(XTemp, XMutationTmp, CR)
        # translate_architecture(XCrossover[0], display=True)

        # 保存每一代个体
        if not os.path.exists("./generation"):
            os.makedirs("./generation") # 如果不存在目录figure_save_path，则创建
        
        for individual in range(NP):
            np.save(f"./generation/{dsid}_{gen}_{individual}.pkl", XCrossover[individual], allow_pickle=True)
            print(f"保存成功./generation/{dsid}_{gen}_{individual}.pkl")
        
        print("完成种群记录")

        XTemp, fitnessVal = selection(XTemp, XCrossover, fitnessVal, dls)
        print("Select完成")
        # calFitness(XCrossover[0])
        best_acc = saveBest(XTemp, fitnessVal, gen, dsid, dls)

        acc_map.append(best_acc)
        # fitness_for_record.append(fitnessVal)
        fitness_for_record = np.append(fitness_for_record, fitnessVal, axis=1)

        print(fitness_for_record)
        test = pd.DataFrame(data=fitness_for_record)
        # print("以下是原始数据")
        # print(test)
        test.to_csv(f'./{dsid}_raw_data.csv',encoding='gbk')
        
        gen += 1
    print(f"Dataset{dsid} detail:")
    print(fitness_for_record)
    # test = pd.DataFrame(data=fitness_for_record)
    # print("以下是原始数据")
    # print(test)
    # test.to_csv(f'./{dsid}_raw_data.csv',encoding='gbk')

    return acc_map




# # 正文代码----------------

if __name__ == '__main__':


    dataset_acc = []
    for dsid in all_dataset:
        print(f"Currently dataset is {dsid}")
        bs = 64
        X, y, splits = get_UCR_data(dsid, return_split=False)
        print(X.shape)
        tfms  = [None, [Categorize()]]
        dsets = TSDatasets(X, y, tfms=tfms, splits=splits)
        
        if dsid in dataset_with_nan:
            batch_transforms = [TSStandardize(by_sample=True), Nan2Value()]
            dls   = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[bs, bs*2], batch_tfms=batch_transforms)
        else:
            dls   = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[bs, bs*2])


        dataset_best_acc = DEA(dsid, dls)

        # 作图
        
        x = np.linspace(1, GEN_MAX + 1, GEN_MAX)
        plt.figure(figsize=(6, 4))# 新建一个图像，设置图像大小
        plt.plot(x, dataset_best_acc, 'ro-', label='DEA')# 设置颜色、标记符号、线型、图例标签
        plt.title(f'Differential Evolution for {dsid}', fontsize=15)# 标题
        plt.xlim(0, GEN_MAX)# x轴范围
        plt.ylim(0, 1)# y轴范围
        plt.xlabel('Generation', fontsize=20)# x轴标签
        plt.ylabel('Best fitness', fontsize=20)# y轴标签
        plt.legend(loc='best')# 图例
        if not os.path.exists("./svg"):
            os.makedirs("./svg") # 如果不存在目录figure_save_path，则创建
        plt.savefig(f'./svg/{dsid}.svg',format='svg')
        plt.show()  

        # 保存此时的raw_data
        dataset_acc.append(dataset_best_acc)
    forsave = pd.DataFrame(data=dataset_acc)
    forsave.to_csv("./overall.csv", encoding='gbk')




    